# 3、练习逻辑回归案例，调整参数，得到最佳精确率及召回率

In [1]:
#逻辑回归

In [2]:
# x=np.arange(0,1,0.01)
# import matplotlib.pyplot as plt
# plt.plot(x,-np.log(x))  #以e为底部
# plt.show()
# x=np.arange(0,1,0.01)
# import matplotlib.pyplot as plt
# plt.plot(x,-np.log(1-x))
# plt.show()

In [7]:
import pandas as pd

"""
逻辑回归做二分类进行癌症预测（根据细胞的属性特征）
:return: NOne
"""
# 构造列标签名字
column = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
          'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli',
          'Mitoses', 'Class']

# 读取数据
data = pd.read_csv(
    "../MachineData/data/breast-cancer-wisconsin.csv",
    names=column)

# print(data) #想看可以解除
#当你读取数据时，看上去是数值的列，读进来是字符串，说明里边
# 存在了非数值情况
print(data.info())
print('*'*100)
print(data.describe(include='all'))
print('*'*100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Sample code number           699 non-null    int64 
 1   Clump Thickness              699 non-null    int64 
 2   Uniformity of Cell Size      699 non-null    int64 
 3   Uniformity of Cell Shape     699 non-null    int64 
 4   Marginal Adhesion            699 non-null    int64 
 5   Single Epithelial Cell Size  699 non-null    int64 
 6   Bare Nuclei                  699 non-null    object
 7   Bland Chromatin              699 non-null    int64 
 8   Normal Nucleoli              699 non-null    int64 
 9   Mitoses                      699 non-null    int64 
 10  Class                        699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB
None
*************************************************************************************************

In [8]:
data['Bare Nuclei'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

In [9]:
import numpy as np
# 缺失值进行处理
print(data.shape)
data = data.replace(to_replace='?', value=np.nan)
#直接删除，哪一行有空值，就删除对应的样本
data = data.dropna()
print('-' * 50)
print(data.shape)

(699, 11)
--------------------------------------------------
(683, 11)


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 683 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Sample code number           683 non-null    int64 
 1   Clump Thickness              683 non-null    int64 
 2   Uniformity of Cell Size      683 non-null    int64 
 3   Uniformity of Cell Shape     683 non-null    int64 
 4   Marginal Adhesion            683 non-null    int64 
 5   Single Epithelial Cell Size  683 non-null    int64 
 6   Bare Nuclei                  683 non-null    object
 7   Bland Chromatin              683 non-null    int64 
 8   Normal Nucleoli              683 non-null    int64 
 9   Mitoses                      683 non-null    int64 
 10  Class                        683 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 64.0+ KB


In [11]:
column[10]

'Class'

In [12]:
#分类的类别是2和4,对应 "良性", "恶性"
data[column[10]].unique()

array([2, 4], dtype=int64)

In [13]:
from sklearn.model_selection import train_test_split
# 进行数据的分割,第零列是编号
x_train, x_test, y_train, y_test = train_test_split(data[column[1:10]], data[column[10]], test_size=0.25,
                                                    random_state=1)
from sklearn.preprocessing import StandardScaler
# 进行标准化处理
std = StandardScaler()

x_train = std.fit_transform(x_train)
x_test = std.transform(x_test)
x_train[0]

array([-1.21629973, -0.70863282, -0.75174943,  0.04301674, -0.55657068,
       -0.71054972, -0.99312055, -0.62911518, -0.36280962])

In [14]:
from sklearn.linear_model import LogisticRegression
import time
from sklearn.metrics import classification_report, roc_auc_score



#
# # 逻辑回归预测
# C正则化力度
# solver = 'liblinear'  solver是学习率优化算法，就是学习率会随着epoch的变化而变化
#solver 分别等于 'liblinear'  'lbfgs'  'newton-cg'  'sag'  'saga'，
#epoch就代表第几次迭代
#max_iter 最大迭代次数

# lg = LogisticRegression(C=0.5, solver='lbfgs')#0.9824561403508771
# lg = LogisticRegression(C=0.5, solver='liblinear')#0.9883040935672515
# lg = LogisticRegression(C=0.5, solver='newton-cg')#0.9824561403508771
# lg = LogisticRegression(C=0.5, solver='sag')#0.9824561403508771
lg = LogisticRegression(C=0.5, solver='saga')# 0.9824561403508771

start= time.time()
lg.fit(x_train, y_train)
end= time.time()
print("逻辑回归的训练时间：", end-start)
# 逻辑回归的权重参数，了解，没那么重要
print('-'*50)
print(lg.coef_)
print('-'*50)
y_predict = lg.predict(x_test)
print(y_predict)
print('-'*50)
print("准确率：", lg.score(x_test, y_test))
print('-'*50)
print(y_test[0:5])
print('-'*50)
print(lg.predict_proba(x_test))  #得出对应分类的概率
print('-'*50)
# 为什么还要看下召回率，labels和target_names对应
# macro avg 平均值  weighted avg 加权平均值
print("召回率：", classification_report(y_test, y_predict, labels=[2, 4], target_names=["良性", "恶性"]))
print('-'*50)
#AUC计算要求是二分类，不需要是0和1
print("AUC指标：", roc_auc_score(y_test, y_predict))
print('-'*50)



逻辑回归的训练时间： 0.004999637603759766
--------------------------------------------------
[[1.11338349 0.25318205 0.79179858 0.6101445  0.07131729 1.10929053
  0.77807397 0.64397888 0.67834428]]
--------------------------------------------------
[2 2 2 4 2 4 2 2 4 4 2 2 4 2 2 4 2 2 2 2 2 2 2 4 4 2 2 2 4 4 2 2 4 4 2 4 2
 2 4 4 4 2 2 4 2 2 2 2 4 2 2 2 4 2 2 2 4 2 2 2 2 4 2 2 2 4 2 4 4 2 2 4 2 2
 4 2 2 2 2 2 2 2 4 2 4 4 2 2 2 4 2 2 4 2 2 4 4 2 2 4 2 2 4 4 2 2 2 2 4 2 4
 4 2 4 2 4 2 2 2 2 4 2 4 2 2 2 2 2 4 2 2 2 2 2 2 2 4 2 4 4 2 2 4 2 2 2 2 4
 2 2 2 2 2 4 2 4 2 4 2 2 4 2 4 2 4 4 2 4 2 2 2]
--------------------------------------------------
准确率： 0.9824561403508771
--------------------------------------------------
444    2
24     2
195    2
49     4
375    2
Name: Class, dtype: int64
--------------------------------------------------
[[9.49048995e-01 5.09510054e-02]
 [9.94950680e-01 5.04931965e-03]
 [9.83690562e-01 1.63094382e-02]
 [2.69271360e-02 9.73072864e-01]
 [9.97326148e-01 2.67385163e-03]
